step 1:  Importing necessary libraries

In [1]:
import numpy as np
from itertools import product
from sklearn.metrics import mean_squared_error
import pandas as pd

step 2: Define a class to represent a node in the decision tree

    Constructor for a tree node.

        Parameters:
        feature_index (int): Index of the feature used for splitting.
        threshold (float): Threshold value for splitting.
        left (Node): Left child node.
        right (Node): Right child node.
        var_red (float): Variance reduction achieved by the split.
        value (float): Value of the leaf node (for predictions).

In [2]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, var_red=None, value=None):
        ''' constructor '''

        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.var_red = var_red

        # for leaf node
        self.value = value





step 3: Define a class for a custom decision tree regressor


In [3]:
class DecisionTreeRegressor():
    def __init__(self, min_samples_split=2, max_depth=2):
        """
        Constructor for a custom decision tree regressor.

        Parameters:
        min_samples_split (int): Minimum number of samples required to split.
        max_depth (int): Maximum depth of the tree.
        """
        self.root = None  # Root node of the tree
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth

    def build_tree(self, dataset, curr_depth=0):
        """
        Recursive function to build the decision tree.

        Parameters:
        dataset (ndarray): Training data including features and target values.
        curr_depth (int): Current depth of the tree.

        Returns:
        Node: A node representing the decision tree structure.
        """
        X, Y = dataset[:, :-1], dataset[:, -1]
        num_samples, num_features = np.shape(X)

        # Check stopping conditions
        if num_samples >= self.min_samples_split and curr_depth <= self.max_depth:
            best_split = self.get_best_split(dataset, num_samples, num_features)
            if best_split and best_split.get("var_red", 0) > 0:
                # Build left and right subtrees recursively
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth + 1)
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth + 1)
                return Node(best_split["feature_index"], best_split["threshold"], left_subtree, right_subtree, best_split["var_red"])

        # Compute and return a leaf node if stopping conditions are met
        leaf_value = self.calculate_leaf_value(Y)
        return Node(value=leaf_value)

    def get_best_split(self, dataset, num_samples, num_features):
        """
        Find the best split for the dataset.

        Parameters:
        dataset (ndarray): Training data including features and target values.
        num_samples (int): Number of samples in the dataset.
        num_features (int): Number of features in the dataset.

        Returns:
        dict: Information about the best split.
        """
        best_split = {}
        max_var_red = -float("inf")

        # Iterate over all features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)

            # Try each possible threshold
            for threshold in possible_thresholds:
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                if len(dataset_left) > 0 and len(dataset_right) > 0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    curr_var_red = self.variance_reduction(y, left_y, right_y)
                    if curr_var_red > max_var_red:
                        best_split = {
                            "feature_index": feature_index,
                            "threshold": threshold,
                            "dataset_left": dataset_left,
                            "dataset_right": dataset_right,
                            "var_red": curr_var_red
                        }
                        max_var_red = curr_var_red

        return best_split

    def split(self, dataset, feature_index, threshold):
        """
        Split the dataset based on a feature and a threshold.

        Parameters:
        dataset (ndarray): Training data.
        feature_index (int): Index of the feature to split on.
        threshold (float): Threshold value for the split.

        Returns:
        tuple: Left and right splits of the dataset.
        """
        dataset_left = np.array([row for row in dataset if row[feature_index] <= threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index] > threshold])
        return dataset_left, dataset_right

    def variance_reduction(self, parent, l_child, r_child):
        """
        Calculate variance reduction achieved by a split.

        Parameters:
        parent (ndarray): Target values of the parent node.
        l_child (ndarray): Target values of the left child.
        r_child (ndarray): Target values of the right child.

        Returns:
        float: Variance reduction.
        """
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        reduction = np.var(parent) - (weight_l * np.var(l_child) + weight_r * np.var(r_child))
        return reduction

    def calculate_leaf_value(self, Y):
        """
        Calculate the value of a leaf node.

        Parameters:
        Y (ndarray): Target values of the samples in the leaf.

        Returns:
        float: Mean of the target values.
        """
        return np.mean(Y)

    def print_tree(self, tree=None, indent=" "):
        """
        Print the decision tree.

        Parameters:
        tree (Node): The root node of the tree (default is None, which means the root of the current tree).
        indent (str): Indentation for tree visualization.
        """
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)
        else:
            print(f"X_{tree.feature_index} <= {tree.threshold}? {tree.var_red}")
            print(f"{indent}left:", end="")
            self.print_tree(tree.left, indent + indent)
            print(f"{indent}right:", end="")
            self.print_tree(tree.right, indent + indent)

    def fit(self, X, Y):
        """
        Train the decision tree.

        Parameters:
        X (ndarray): Feature matrix.
        Y (ndarray): Target values.
        """
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)

    def make_prediction(self, x, tree):
        """
        Make a prediction for a single sample.

        Parameters:
        x (ndarray): Feature vector of the sample.
        tree (Node): The root node of the tree.

        Returns:
        float: Predicted value.
        """
        if tree.value is not None:
            return tree.value
        feature_val = x[tree.feature_index]
        if feature_val <= tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

    def predict(self, X):
        """
        Predict target values for multiple samples.

        Parameters:
        X (ndarray): Feature matrix.

        Returns:
        list: Predicted values.
        """
        return [self.make_prediction(x, self.root) for x in X]


step 4: Define a class for Ensemble Regression Tree


In [4]:
class EnsembleRegressionTree:
    def __init__(self, n_trees=5, min_samples_split=10, learning_rate=0.1, max_depth=2):
        """
        Constructor for Ensemble Regression Tree (Gradient Boosted Trees).

        Parameters:
        n_trees (int): Number of trees in the ensemble.
        min_samples_split (int): Minimum number of samples required to split a node.
        learning_rate (float): Learning rate for the gradient boosting algorithm.
        max_depth (int): Maximum depth of each tree.
        """
        self.n_trees = n_trees
        self.min_samples_split = min_samples_split
        self.learning_rate = learning_rate
        self.trees = []  # List to store individual trees
        self.initial_prediction = None  # Initial prediction (mean of the target values)
        self.max_depth = max_depth

    def fit(self, X, y):
        """
        Train the ensemble regression tree using gradient boosting.

        Parameters:
        X (ndarray): Feature matrix.
        y (ndarray): Target values.
        """
        # Initialize the prediction as the mean of the target values
        self.initial_prediction = np.mean(y)
        predictions = np.full(y.shape, self.initial_prediction)

        for i in range(self.n_trees):
            # Calculate residuals (difference between actual and predicted values)
            residuals = y.ravel() - predictions.ravel()
            print(f"Tree {i + 1}: residuals shape = {residuals.shape}, predictions shape = {predictions.shape}")

            # Train a new tree to predict the residuals
            tree = DecisionTreeRegressor(min_samples_split=self.min_samples_split, max_depth=self.max_depth)
            tree.fit(X, residuals.reshape(-1, 1))  # Fit the tree with residuals
            self.trees.append(tree)

            # Update predictions with the current tree's output scaled by the learning rate
            predictions += self.learning_rate * np.array(tree.predict(X)).reshape(-1, 1)

    def predict(self, X):
        """
        Predict target values for new data using the trained ensemble.

        Parameters:
        X (ndarray): Feature matrix.

        Returns:
        ndarray: Predicted values.
        """
        predictions = np.full((X.shape[0], 1), self.initial_prediction)
        for tree in self.trees:
            predictions += self.learning_rate * np.array(tree.predict(X)).reshape(-1, 1)
        return predictions.ravel()  # Return predictions as a 1D array


    def print_tree(self):
        print(f"Initial prediction: {self.initial_prediction}")
        for i, tree in enumerate(self.trees):
            print(f"\nTree {i + 1}:")
            tree.print_tree()

step 5: Hyperparameter tuning for ensemble regression tree


In [5]:
def custom_hyperparameter_tuning(X_train, y_train, X_test, y_test):
    """
    Perform hyperparameter tuning for a custom decision tree regressor.

    Parameters:
    X_train (ndarray): Training feature matrix.
    y_train (ndarray): Training target values.
    X_test (ndarray): Testing feature matrix.
    y_test (ndarray): Testing target values.

    Returns:
    tuple: Best hyperparameters and the corresponding MSE.
    """
    max_depth_values = [5, 10, 15]  # Possible values for maximum depth
    min_samples_split_values = [5, 10, 20]  # Possible values for minimum samples to split
    best_params = None
    best_mse = float('inf')

    # Grid search over the hyperparameter space
    for min_samples_split, max_depth in product(min_samples_split_values, max_depth_values):
        print(f"Training model: min_samples_split={min_samples_split}, max_depth={max_depth}")
        tree = DecisionTreeRegressor(min_samples_split=min_samples_split, max_depth=max_depth)
        tree.fit(X_train, y_train)
        y_pred = tree.predict(X_test)  # Predictions on test set
        mse = mean_squared_error(y_test, y_pred)  # Compute MSE
        print(f"MSE: {mse}")

        # Update best parameters if current MSE is lower
        if mse < best_mse:
            best_mse = mse
            best_params = {'min_samples_split': min_samples_split, 'max_depth': max_depth}

    print("Best parameters:", best_params)
    print("Best MSE:", best_mse)
    return best_params, best_mse


step 6: Hyperparameter tuning for ensemble regression tree

In [6]:
def ensemble_hyperparameter_tuning(X_train, y_train, X_test, y_test):
    """
    Perform hyperparameter tuning for an ensemble regression tree.

    Parameters:
    X_train (ndarray): Training feature matrix.
    y_train (ndarray): Training target values.
    X_test (ndarray): Testing feature matrix.
    y_test (ndarray): Testing target values.

    Returns:
    tuple: Best hyperparameters and the corresponding MSE.
    """
    n_trees_values = [5]  # Possible number of trees in the ensemble
    learning_rate_values = [0.1, 0.2, 0.3, 0.5]  # Possible learning rates
    max_depth_values = [5, 10, 15]  # Possible values for maximum depth
    best_params = None
    best_mse = float('inf')

    # Grid search over the hyperparameter space
    for n_trees, learning_rate, max_depth in product(n_trees_values, learning_rate_values, max_depth_values):
        print(f"Training model: n_trees={n_trees}, learning_rate={learning_rate}, max_depth={max_depth}")
        ensemble = EnsembleRegressionTree(n_trees=n_trees, learning_rate=learning_rate, max_depth=max_depth)
        ensemble.fit(X_train, y_train)
        y_pred = ensemble.predict(X_test)  # Predictions on test set
        mse = mean_squared_error(y_test, y_pred)  # Compute MSE
        print(f"MSE: {mse}")

        # Update best parameters if current MSE is lower
        if mse < best_mse:
            best_mse = mse
            best_params = {'n_trees': n_trees, 'learning_rate': learning_rate, 'max_depth': max_depth}

    print("Best parameters:", best_params)
    print("Best MSE:", best_mse)
    return best_params, best_mse


step 7: Main program for running the custom and ensemble regression tree models

In [7]:
# Main program for running the custom and ensemble regression tree models
if __name__ == "__main__":
    # Load and preprocess data (assumed to be already completed)
    train = pd.read_csv('train_x.csv')
    trainGT = pd.read_csv('train_y.csv')
    test = pd.read_csv('test_x.csv')
    testGT = pd.read_csv('test_y.csv')

    # Extract features and targets from the datasets
    X_train = train.iloc[:, :-1].values
    X_test = test.iloc[:, :-1].values
    y_train = trainGT.iloc[:, -1].values.reshape(-1, 1)  # Reshape to 2D
    y_test = testGT.iloc[:, -1].values.reshape(-1, 1)  # Reshape to 2D

    print("===== Hyperparameter Tuning for Custom Decision Tree =====")
    best_tree_params, best_tree_mse = custom_hyperparameter_tuning(X_train, y_train, X_test, y_test)

    print("\n===== Hyperparameter Tuning for Ensemble Regression Tree =====")
    best_ensemble_params, best_ensemble_mse = ensemble_hyperparameter_tuning(X_train, y_train, X_test, y_test)

    print(f"X_train shape: {X_train.shape}")
    print(f"y_train shape: {y_train.shape}")

    # Check and reshape y_train to ensure compatibility
    if len(y_train.shape) == 1:
        y_train = y_train.reshape(-1, 1)

    print(f"y_train reshaped: {y_train.shape}")

    # Train and compare sklearn decision tree and custom decision tree
    sklearn_tree = DecisionTreeRegressor(min_samples_split=10)
    sklearn_tree.fit(X_train, y_train)

    my_tree = DecisionTreeRegressor(min_samples_split=10)
    my_tree.fit(X_train, y_train)

    # Predictions for sklearn and custom decision trees
    y_pred_sklearn = sklearn_tree.predict(X_test)
    y_pred_custom = my_tree.predict(X_test)




===== Hyperparameter Tuning for Custom Decision Tree =====
Training model: min_samples_split=5, max_depth=5
MSE: 0.5620161979447281
Training model: min_samples_split=5, max_depth=10
MSE: 0.5666960085633368
Training model: min_samples_split=5, max_depth=15
MSE: 0.6332245604057446
Training model: min_samples_split=10, max_depth=5
MSE: 0.5620161979447281
Training model: min_samples_split=10, max_depth=10
MSE: 0.5599178463104962
Training model: min_samples_split=10, max_depth=15
MSE: 0.6236392206886414
Training model: min_samples_split=20, max_depth=5
MSE: 0.5618135315018532
Training model: min_samples_split=20, max_depth=10
MSE: 0.5404291842557926
Training model: min_samples_split=20, max_depth=15
MSE: 0.5885212599166756
Best parameters: {'min_samples_split': 20, 'max_depth': 10}
Best MSE: 0.5404291842557926

===== Hyperparameter Tuning for Ensemble Regression Tree =====
Training model: n_trees=5, learning_rate=0.1, max_depth=5
Tree 1: residuals shape = (5196,), predictions shape = (5196,

In [8]:
    # Train ensemble regression tree with specific parameters: {'n_trees': 5, 'learning_rate': 0.5, 'max_depth': 10}
    print("\n===== Training Ensemble Regression Tree with Fixed Parameters =====")
    ensemble = EnsembleRegressionTree(n_trees=5, learning_rate=0.3, max_depth=10)
    ensemble.fit(X_train, y_train)
    y_pred_ensemble = ensemble.predict(X_test)

    # Print predictions and performance comparison
    print("\n===== Model Performance =====")
    print("MSE (sklearn):", mean_squared_error(y_test, y_pred_sklearn))
    print("MSE (custom):", mean_squared_error(y_test, y_pred_custom))
    """

    From the previous results, we observed that while the lowest MSE 
    was achieved with learning_rate=0.5 and max_depth=15, the combination 
    of a higher learning rate and maximum depth increases the risk of overfitting. 
    Using learning_rate=0.3 and max_depth=10, the model achieved a very similar MSE 
    with a simpler structure and reduced risk of overfitting. Therefore, we have 
    decided to set the max_depth to 10 and learning_rate to 0.3, balancing model complexity and performance.
    """
    print("MSE (ensemble with parameters {'n_trees': 5, 'learning_rate': 0.3, 'max_depth': 10}):", 
          mean_squared_error(y_test, y_pred_ensemble))


===== Training Ensemble Regression Tree with Fixed Parameters =====
Tree 1: residuals shape = (5196,), predictions shape = (5196, 1)
Tree 2: residuals shape = (5196,), predictions shape = (5196, 1)
Tree 3: residuals shape = (5196,), predictions shape = (5196, 1)
Tree 4: residuals shape = (5196,), predictions shape = (5196, 1)
Tree 5: residuals shape = (5196,), predictions shape = (5196, 1)

===== Model Performance =====
MSE (sklearn): 0.6206453857772722
MSE (custom): 0.6206453857772722
MSE (ensemble with parameters {'n_trees': 5, 'learning_rate': 0.3, 'max_depth': 10}): 0.4417771598466974


print tree structure

In [15]:
    print("-------my_tree structure-------")
    my_tree.print_tree()

    print("-------ensemble_tree structure-------")
    ensemble.print_tree()



-------my_tree structure-------
X_7 <= 0.99223? 0.08164085750697447
 left:X_5 <= 9.0? 0.06628350345864265
  left:X_3 <= 5.0? 0.14779452276778493
    left:5.019607843137255
    right:6.5
  right:X_3 <= 1.75? 0.046829268217513875
    left:6.142589118198874
    right:6.578078078078078
 right:X_2 <= 0.26? 0.03533327998780045
  left:X_1 <= 0.23? 0.020497270819085478
    left:5.784090909090909
    right:5.35303776683087
  right:X_1 <= 0.205? 0.026929128036374816
    left:6.130693069306931
    right:5.719040626529614
-------ensemble_tree structure-------
Initial prediction: 5.822748267898383

Tree 1:
X_7 <= 0.99223? 0.08164085750697447
 left:X_5 <= 9.0? 0.06628350345864265
  left:X_3 <= 5.0? 0.14779452276778493
    left:X_9 <= 0.53? 0.1788220988349739
        left:X_8 <= 3.27? 0.12304111968795561
                left:X_8 <= 3.16? 0.1411107118617002
                                left:X_4 <= 0.034? 0.1297596543343234
                                                                left:X_3 <= 